In [23]:
from playwright.async_api import async_playwright
# Update python path to include the parent directory
import os
import sys
import datetime
from datetime import timedelta

from urllib import parse

sys.path.append("..")

os.environ["ROOT_PATH"] = "/Users/aennassiri/Projects/tickettakedown/app"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/aennassiri/Projects/tickettakedown/app/configuration/fubloo-app-1f213ca274de.json"
)

In [15]:
import requests

import pandas as pd
from src.services.leads import get_leads, get_last_lead

/var/folders/kb/yw9y12616lz4f5jp49qvpmmm0000gn/T/ipykernel_61225/3310728289.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)
# browser = await pw.chromium.launch()


In [7]:
import os
os.makedirs('playwright/.auth', exist_ok=True)
if os.path.exists('playwright/.auth/state.json'):
    context = await browser.new_context(storage_state="playwright/.auth/state.json")
    page = await context.new_page()


In [36]:
async def extract_info(page):
    container = await page.query_selector('.e1usz6i03')
    element = await container.query_selector('#phone-number-content')
    phone_number  = await element.inner_text()
    print(phone_number)
    element = await container.query_selector("#email-address-content")
    email_address = await element.inner_text()
    print(email_address)
    element = await container.query_selector("#past-addresses-content")
    past_addresses = await element.inner_text()
    print(past_addresses)
    element = await container.query_selector("#current-address-content")
    current_address = await element.inner_text()
    print(current_address)
    element = await container.query_selector("#summary-name")
    summary_name = await element.inner_text()
    print(summary_name) 
    return {
        "phone_number": phone_number,
        "email_address": email_address,
        "past_addresses": past_addresses,
        "current_address": current_address,
        "summary_name": summary_name
    }

In [42]:
async def search_person(page, firstName, lastName, middleName=None, age=None,
    city=None,
    state=None,
    zip=None,
    addressLine1=None,
    addressLine2=None):
    url = f"https://www.spokeo.com/{firstName}-{lastName}"
    params = {}
    if middleName is not None:
        params["middle_name"] = middleName
    if age is not None:
        # age ranges: 18-25; 26-39; 40-49; 50-59; 60+
        if age < 18:
            raise ValueError("Age must be greater than 18")
        if age < 26:
            params["age_range"] = "18-25"
        elif age < 40:
            params["age_range"] = "26-39"
        elif age < 50:
            params["age_range"] = "40-49"
        elif age < 60:
            params["age_range"] = "50-59"
        else:
            params["age_range"] = "60+"
    if city is not None:
        params["city"] = city
    if state is not None:
        params["state"] = state
    if params:
        url = f"{url}?{parse.urlencode(params)}"

    print(url)
    await page.goto(url)    

    link_elements = await page.query_selector_all('.e1ndw42t0')
    # links = [
    #     link_element.get_attribute('href')
    #     for link_element in link_elements
    # ]
    links = []
    for link_element in link_elements:
        link = await link_element.get_attribute('href')
        links.append(f"https://www.spokeo.com{link}")

    results = []
    for link in links:
        await page.goto(link)
        try:
            output = await extract_info(page)
            results.append(output)
        except Exception as e:
            print(e,"==>", link)
            pass

    return results

# Leads Analysis


In [43]:
today = datetime.datetime.now()

leads_not_found = get_last_lead(
    start_date=today - timedelta(days=5),
    end_date=today + timedelta(days=1),
    status="new",
    limit=100,
)

df = pd.DataFrame([lead.model_dump() for lead in leads_not_found])


df["middle_name"] = df["middle_name"].fillna("")
df["first_name"] = df["first_name"].fillna("")
df["last_name"] = df["last_name"].fillna("")
df["state"] = df["state"].fillna("MO")

/Users/aennassiri/Projects/tickettakedown/app/src/services/leads.py:145: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", ">=", start_date)
/Users/aennassiri/Projects/tickettakedown/app/src/services/leads.py:149: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", "<=", end_date)
/Users/aennassiri/Projects/tickettakedown/app/src/services/leads.py:151: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("status", "==", status)


In [44]:
df

,id,case_id,first_name,last_name,middle_name,court_code,age,year_of_birth,email,phone,...,notes,violation,court,accidentCheckbox,commercialDriverLicence,ticket_img,user_id,report,details,cloudtalk_upload
0,None,230013210,WYATT,FREDRICKSON,MATTHEW,CT16,19.0,2004.0,,None,...,None,None,None,False,False,230013210.png,None,None,,False
1,None,230013207,BRETT,COX,KEITH,CT16,25.0,1998.0,,None,...,None,None,None,False,False,230013207.png,None,None,,False
2,None,230013184,SETH,CALDWELL,ANDREW,CT16,41.0,1982.0,,None,...,None,None,None,False,False,230013184.png,None,None,,False
3,None,230013168,RYAN,BREIT,E,CT16,46.0,1977.0,,None,...,None,None,None,False,False,230013168.png,None,None,,False
4,None,230013158,RAFIQ,MCDANIEL,JAMAL,CT16,44.0,1979.0,,None,...,None,None,None,False,False,230013158.png,None,None,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,None,160072213,LAURIANO,CHAVEZ,,CT31,37.0,1986.0,,None,...,None,None,None,False,False,160072213.png,None,None,,False
765,None,006214227,HASHEEM,PARKER,M,CT16,29.0,1994.0,,None,...,None,None,None,False,False,006214227.png,None,None,,False
766,None,000006347,CLAUDE,SIMMONS,,CT21,53.0,1970.0,,None,...,None,None,None,False,False,000006347.png,None,None,,False
767,None,000006346,CLAUDE,SIMMONS,,CT21,53.0,1970.0,,None,...,None,None,None,False,False,000006346.png,None,None,,False


In [45]:
results = {}

for index, row in df.iterrows():
    print(f"Looking for {row['case_id']}")
    print(
        row["first_name"],
        row["last_name"],
        row["middle_name"],
        row["age"],
        row["city"],
        row["state"],
        row["zip_code"],
        row["address"],
    )
    results = await search_person(
        page,
        row["first_name"],
        row["last_name"],
        row["middle_name"],
        row["age"],
        row["city"],
        row["zip_code"],
        row["state"],
        row["address"],
    )

Looking for 230013210
WYATT FREDRICKSON MATTHEW 19.0 None MO None None
https://www.spokeo.com/WYATT-FREDRICKSON?middle_name=MATTHEW&age_range=18-25
See available information
fub_bucket@hotmail.com
See available information
5772 W North St, Clifton, ID
Wyatt Fredrickson, Age 74
(281) 373-4246
wyattfredrickson@icloud.com
See available information
26802 Temple Park Ln, Cypress, TX
Wyatt Fredrickson
Looking for 230013207
BRETT COX KEITH 25.0 None MO None None
https://www.spokeo.com/BRETT-COX?middle_name=KEITH&age_range=18-25
(606) 836-6104 +2 phones
wbcox50@yahoo.com +1 email
114 Patton Dr Ashland, KY +7 more
1103 Green Acres Dr, Russell, KY
Brett A Cox, Age 35
(770) 233-8959 +1 phone
See available information
1276 Geisky Creek Rd Hayesville, NC +10 more
621 Drewery Ln, Griffin, GA
Brett Jerome Cox, Age 62
(207) 685-9132 +5 phones
goodkid@inbox.com +1 email
26 Liberty Ln Readfield, ME +5 more
3 Captain Sawyer Ln, Kennebunk, ME
Brett W Cox, Age 51
(303) 495-2323 +3 phones
See available info

CancelledError: 

In [41]:
results

[{'phone_number': '(202) 486-1318 +1 phone',
  'email_address': 'broadway_81@hotmail.com',
  'past_addresses': '4311 N Ozark Ave Harwood Heights, IL +6 more',
  'current_address': '3304 W Schubert Ave, Apt 2I, Chicago, IL',
  'summary_name': 'Sarah E Spies, Age 42'}]

In [265]:
for index, row in df.iterrows():
    try:
        result = await preson_search(page, row['first_name'], row['last_name'], row['middle_name'])
        print(f"user {row['first_name']} ==> ", result)
    except Exception as e:
        print(e)
        result = []

    phone_numbers_spokeo.append(result)

'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/California/Garden-Grove/p2018020712336929942701038168173
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/Texas/Austin/p2016110406511622702879694864356
user JOHN ==>  ['(714) 894-6399', '(803) 356-1433 +4 phones', '(480) 839-4121 +3 phones', '(803) 371-9060 +4 phones', '(904) 744-7825 +4 phones', '(803) 822-8832 +6 phones', '(302) 258-8358 +1 phone', '(318) 934-3491 +3 phones', '(435) 729-9560 +5 phones', '(803) 920-0165 +8 phones', '(423) 383-8624 +7 phones', '(513) 594-7331 +5 phones', '(770) 808-4510', '(973) 627-6102', '(302) 258-8999 +2 phones', '(714) 850-0688', '(973) 839-2535', '(770) 386-9157', '(402) 758-1233 +1 phone', '(317) 796-4992 +1 phone', '(440) 821-3073 +4 phones', '(712) 534-3549', '(540) 478-3578 +7 phones', '(315) 662-8046 +3 phones', '(423) 383-8624 +2 phones', '(919) 868-1573 +1 phone', '(318) 949-2063 +1 phone', 'See availab

In [266]:
phone_numbers_spokeo

[['(714) 894-6399',
  '(803) 356-1433 +4 phones',
  '(480) 839-4121 +3 phones',
  '(803) 371-9060 +4 phones',
  '(904) 744-7825 +4 phones',
  '(803) 822-8832 +6 phones',
  '(302) 258-8358 +1 phone',
  '(318) 934-3491 +3 phones',
  '(435) 729-9560 +5 phones',
  '(803) 920-0165 +8 phones',
  '(423) 383-8624 +7 phones',
  '(513) 594-7331 +5 phones',
  '(770) 808-4510',
  '(973) 627-6102',
  '(302) 258-8999 +2 phones',
  '(714) 850-0688',
  '(973) 839-2535',
  '(770) 386-9157',
  '(402) 758-1233 +1 phone',
  '(317) 796-4992 +1 phone',
  '(440) 821-3073 +4 phones',
  '(712) 534-3549',
  '(540) 478-3578 +7 phones',
  '(315) 662-8046 +3 phones',
  '(423) 383-8624 +2 phones',
  '(919) 868-1573 +1 phone',
  '(318) 949-2063 +1 phone',
  'See available information'],
 ['(910) 461-0094 +5 phones',
  '(402) 812-1311 +7 phones',
  '(618) 724-7335',
  '(812) 882-9223',
  'See available information',
  '(303) 884-3599 +3 phones',
  '(540) 797-5004 +7 phones',
  '(901) 233-1586 +4 phones'],
 ['(714) 67

In [267]:
df['phone_numbers_spokeo'] = phone_numbers_spokeo


In [276]:
df['phone_numbers_spokeo'][4]


[]

In [269]:
df.to_csv('phone_numbers_spokeo.csv', index=False)

In [270]:
await browser.close()
await pw.stop()